In [16]:
import faker
import random
import pandas as pd
from datetime import date
from datetime import timedelta
import numpy as np

In [17]:
def generador_dni():
    POSSIBLE_LETTERS = ("T","R","W","A","G","M","Y","F","P","D","X","B","N","J","Z","S","Q","V","H","L","C","K","E","T")
    NUMBER_DNI = random.randint(10000000, 99999999)
    LETTER_DNI = POSSIBLE_LETTERS[NUMBER_DNI % 23]
    return f"{NUMBER_DNI}{LETTER_DNI}"

In [18]:
def edad(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [19]:
psicologos = ['Elena', 'Oriol', 'Psicologo3']
n_patients = 10

In [20]:
fake = faker.Faker('es_ES')

# Generate patient list

In [21]:
nombres = []
nacimientos = []
padres = []
dnis = []
telefonos1 = []
telefonos2 = []
direcciones = []
mails = []
derivantes = []
profesionales = []
tratamientos = []
inicio_tratamientos = []
notas = []

for _ in range(n_patients):
    profile = fake.profile()
    nombres.append(profile['name'])
    nacimientos.append(profile['birthdate'])
    dnis.append(generador_dni())
    direcciones.append(profile['address'])
    mails.append(profile['mail'])
    derivantes.append('medico{}'.format(random.randint(0, 5)))
    profesionales.append(random.choice(psicologos))
    tratamientos.append('terapia')
    
    telefonos1.append(random.randint(600000000,699999999))
    
    if random.randint(0, 10) > 5:
        telefonos2.append(random.randint(600000000,699999999))
        notas.append(profile['job'])
    else:
        telefonos2.append(None)
        notas.append(None)
    
    if edad(nacimientos[-1]) < 18:
        padre = fake.profile()
        padres.append(padre['name'])
    else:
        padres.append(None)
        
    inicio_tratamientos.append(fake.profile()['birthdate'])
    

In [22]:
df_pacientes = pd.DataFrame({'Nombre' : nombres,
                             'Fecha nacimiento' : nacimientos,
                             'Padre/Madre (si menor)' : padres,
                             'DNI' : dnis,
                             'Telefono 1' : telefonos1,
                             'Telefono 2' : telefonos2,
                             'Direccion' : direcciones,
                             'Mail' : mails,
                             'Profesional' : profesionales,
                             'Tipo de tratamiento' : tratamientos,
                             'Inicio tratamiento' : inicio_tratamientos,
                             'Notas' : notas})

In [23]:
df_pacientes.to_excel('listado_pacientes.xlsx', index=False)

# Generate monthly visits

In [24]:
def generate_random_visit_date(month, year):
    start_date = date(year, month  , 1)
    end_date   = date(year, month+1, 1)

    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + timedelta(days=random_number_of_days)
    return random_date

In [25]:
def generate_random_monthly_visits(df_pacientes, month, year):
    fechas        = []
    profesionales = []
    pacientes     = []
    dnis          = []
    conceptos     = []
    euros         = []

    patients = (np.random.rand(n_patients) > 0.4).nonzero()[0]
    for patient in patients:
        datos_paciente = df_pacientes.loc[patient]
        n_visits       = random.randint(1, 11)
        for visit in range(n_visits):
            fechas       .append(generate_random_visit_date(month, year))
            profesionales.append(datos_paciente['Profesional'])
            pacientes    .append(datos_paciente['Nombre'])
            dnis         .append(datos_paciente['DNI'])
            conceptos    .append(datos_paciente['Tipo de tratamiento'])
            euros        .append(random.randint(50, 80))
            
    df_visitas = pd.DataFrame({'Fecha' : fechas,
                               'Profesional' : profesionales,
                               'Paciente' : pacientes,
                               'DNI' : dnis,
                               'Concepto' : conceptos,
                               'Euros' : euros})
    
    return df_visitas

In [27]:
year = 2022

start_month = 1
end_month = 5

for month in range(start_month, end_month):
    df_visitas = generate_random_monthly_visits(df_pacientes, month, year)
    print(f"{month}-{year}\t{df_visitas.shape[0]} visits")
    filename = 'visitas_{}_{}.xlsx'.format(month, year)
    df_visitas.to_excel(filename, index=False)

1-2022	46 visits
2-2022	37 visits
3-2022	33 visits
4-2022	24 visits
